In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Koushik Sripathi\\Desktop\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Koushik Sripathi\\Desktop\\Chicken-Disease-Classification'

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [46]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [47]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [48]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_cpkt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_cpkt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        
        return prepare_callbacks_config

In [49]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_cpkt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_cpkt_callbacks(self):
        return [
            self.create_tb_callbacks,
            self.create_cpkt_callbacks
        ]

In [51]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_cpkt_callbacks()
    
except Exception as e:
    raise e

[2025-10-01 19:45:46,481 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-10-01 19:45:46,483 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-10-01 19:45:46,483 : INFO : common : created directory at: artifacts]
[2025-10-01 19:45:46,484 : INFO : common : created directory at: artifacts\prepare_callbacks\checkpoint]
[2025-10-01 19:45:46,484 : INFO : common : created directory at: artifacts\prepare_callbacks\tensorboard_root_log_dir]


In [1]:
import time 
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
timestamp

'2025-10-03-19-40-19'